In [13]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import requests

In [14]:
FEATURES = ['Socioeconomic Score', 'Study Hours', 'Sleep Hours', 'Attendance (%)', 'Grades']

df = pd.read_csv('data/data.csv')

df = df[FEATURES]

In [15]:
df.head()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,3.4,8.2,53.0,47.0
1,0.85566,3.2,5.9,55.0,35.0
2,0.68025,3.2,9.3,41.0,32.0
3,0.25936,3.2,8.2,47.0,34.0
4,0.60447,3.8,10.0,75.0,33.0


In [16]:
df.describe()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
count,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000
mean,0.552274,4.560807,8.047262,58.536023,40.691643
std,0.261272,1.897581,1.370700,11.675287,9.467358
min,0.101280,0.800000,4.800000,40.000000,32.000000
25%,0.322118,3.475000,7.000000,49.000000,34.000000
50%,0.545945,3.900000,8.400000,57.000000,35.000000
75%,0.789610,5.900000,9.100000,66.000000,47.000000
max,0.999820,10.000000,10.000000,100.000000,91.000000


In [17]:
scaler = MinMaxScaler()

MIN = df[FEATURES[4]].min()
MAX = df[FEATURES[4]].max()

In [18]:
MIN, MAX

(32.0, 91.0)

In [19]:
def preprocessing_fn(inputs):
    # Normalize the inputs
    inputs = scaler.fit_transform(inputs)
    return inputs

In [20]:
df[FEATURES[0]] = df[FEATURES[0]]

df[FEATURES[1:4]] = preprocessing_fn(df[FEATURES[1:4]])

df.head()

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,0.282609,0.653846,0.216667,47.0
1,0.85566,0.260870,0.211538,0.250000,35.0
2,0.68025,0.260870,0.865385,0.016667,32.0
3,0.25936,0.260870,0.653846,0.116667,34.0
4,0.60447,0.326087,1.000000,0.583333,33.0


In [21]:
test_data = df.iloc[4]

test_data

Socioeconomic Score     0.604470
Study Hours             0.326087
Sleep Hours             1.000000
Attendance (%)          0.583333
Grades                 33.000000
Name: 4, dtype: float64

In [22]:
def prepare_json(data_student):
    return {
        "signature_name": "serving_default",
        "instances": [
            {
                "normalized_socioeconomic_score": [data_student['Socioeconomic Score']],
                "normalized_sleep_hours": [data_student['Sleep Hours']],
                "normalized_study_hours": [data_student['Study Hours']],
                "normalized_attendance": [data_student['Attendance (%)']],
            }
        ]
    }

In [23]:
def make_predictions(test_data):
    json_data = prepare_json(test_data)

    url = "http://localhost:8080/v1/models/predict-student-performance-model:predict"
    response = requests.post(url, json=json_data)

    y_pred_normalized = response.json()["predictions"][0][0]

    prediction = y_pred_normalized * (MAX - MIN) + MIN

    return int(prediction)

In [24]:
make_predictions(test_data)

34